# Compare RAPIDS Dimension Reduction Techniques
Heavily influenced by https://umap-learn.readthedocs.io/en/latest/auto_examples/plot_algorithm_comparison.html#sphx-glr-auto-examples-plot-algorithm-comparison-py

## Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import dask_cudf
import cudf
import cuml
import pandas as pd
from sklearn import datasets, decomposition, manifold, preprocessing
from colorsys import hsv_to_rgb

## Prepare and size your data

Things to note about the datasets:

- Blobs: A set of five gaussian blobs in 10 dimensional space. This should be a prototypical example of something that should clearly separate even in a reduced dimension space.
- Iris: a classic small dataset with one distinct class and two classes that are not clearly separated.
- Digits: handwritten digits – ideally different digit classes should form distinct groups. Due to the nature of handwriting digits may have several forms (crossed or uncrossed sevens, capped or straight line oes, etc.)
- Wine: wine characteristics ideally used for a toy regression. Ultimately the data is essentially one dimensional in nature.
- Swiss Roll: data is essentially a rectangle, but has been “rolled up” like a swiss roll in three dimensional space. Ideally a dimension reduction technique should be able to “unroll” it. The data has been coloured according to one dimension of the rectangle, so should form a rectangle of smooth color variation.
- Sphere: the two dimensional surface of a three dimensional sphere. This cannot be represented accurately in two dimensions without tearing. The sphere has been coloured with hue around the equator and black to white from the south to north pole.

In [ ]:
## Just for kicks, we 10xed the samples compared to the original source code.  Cause we can.  
sns.set(context="paper", style="white")

blobs, blob_labels = datasets.make_blobs(
    n_samples=5000, n_features=10, centers=5, random_state=42
)
iris = datasets.load_iris()
digits = datasets.load_digits(n_class=10)
wine = datasets.load_wine()
swissroll, swissroll_labels = datasets.make_swiss_roll(
    n_samples=10000, noise=0.1, random_state=42
)
sphere = np.random.normal(size=(600, 3))
sphere = preprocessing.normalize(sphere)
sphere_hsv = np.array(
    [
        (
            (np.arctan2(c[1], c[0]) + np.pi) / (2 * np.pi),
            np.abs(c[2]),
            min((c[2] + 1.1), 1.0),
        )
        for c in sphere
    ]
)
sphere_colors = np.array([hsv_to_rgb(*c) for c in sphere_hsv])

## Call your algorithms and define your iterations

In [ ]:
## Change your parameters so that you can see how it affects your results
nc = 2
rs = 42
nn = 30

## Iterate through our decomposition algorithms TSVD, PCA, and UMAP
reducers = [
    (cuml.TruncatedSVD(n_components=nc,algorithm='full', random_state=rs)),
    (cuml.TruncatedSVD(n_components=nc,algorithm='jacobi', random_state=rs)),
    (cuml.PCA(n_components=nc,svd_solver='full',whiten=False, random_state=rs)),
    (cuml.PCA(n_components=nc,svd_solver='jacobi',whiten=False, random_state=rs)),
    (cuml.UMAP(n_neighbors=nn, init="spectral")),
    (cuml.TSNE(n_components = nc, method = 'barnes_hut')),
]

##  Iterate through your datasets
test_data = [
    (blobs, blob_labels),
    (iris.data, iris.target),
    (digits.data, digits.target),
    (wine.data, wine.target),
    (swissroll, swissroll_labels),
    (sphere, sphere_colors),
]

## Name your data
dataset_names = ["Blobs", "Iris", "Digits", "Wine", "Swiss Roll", "Sphere"]

## Helper variables
n_rows = len(test_data)
n_cols = len(reducers)
ax_index = 1
ax_list = []

## Size your plots
plt.rcParams["figure.figsize"] = [20,20]
plt.subplots_adjust(
    left=.2, right=10, bottom=.001, top=.96, wspace=.05, hspace=.1
)


## Run your tests

In [ ]:
%%time
for data, labels in test_data:
    gdf=cudf.DataFrame.from_records(data) # skLearn data is a numpy ndarray, so we can just use "from_records" to put it into a cudf dataframe"
    for reducer in reducers:
        start_time = time.time()
        embedding = reducer.fit_transform(gdf)
        elapsed_time = time.time() - start_time
        ax = plt.subplot(n_rows, n_cols, ax_index)
        #print(embedding.T)
        embedding_numpy = embedding.to_pandas().values
        #pdb.set_trace()
        if isinstance(labels[0], tuple):
            ax.scatter(*embedding_numpy.T, s=10, c=labels, alpha=0.5)
        else:
            ax.scatter(
                *embedding_numpy.T, s=10, c=labels, cmap="Spectral", alpha=0.5
            )
        ax.text(
            0.99,
            0.01,
            "{:.2f} s".format(elapsed_time),
            transform=ax.transAxes,
            size=14,
            horizontalalignment="right",
        )
        ax_list.append(ax)
        ax_index += 1
plt.setp(ax_list, xticks=[], yticks=[])
for i in np.arange(n_rows) * n_cols:
    ax_list[i].set_ylabel(dataset_names[i // n_cols], size=16)
for i in range(n_cols):
    ax_list[i].set_xlabel(repr(reducers[i]).split(".")[0].split("(")[0], size=16)
    ax_list[i].xaxis.set_label_position("top")

#plt.tight_layout()
plt.show()

In [ ]:
repr(reducers[2]).split(".")[0].split("(")[0]

Checking out the images, you can see how different dimension reduction methods will cluster, represent your data, and how fast it will do it.  This enables you to pick the best one for your uses.  You can change `rs`, `nc`, `nn` to see how the decomposition changes using the visualizations.